In [1]:
'''Cell 1: Imports and Setup'''
# ==============================================================================
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

print("Imports complete.")

Imports complete.


In [2]:
'''Cell 2: Configuration and Path Setup'''
# ==============================================================================
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
RAW_ROOT_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')
INTERIM_ROOT_DIR = os.path.join(PROJECT_ROOT, 'data', 'interim')
METADATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'raw', 'Coupon_metadata.csv') # State metadata table csv here

BOX_SIZE = 14.0
ANGLE_DEG = 10

print(f"Project Root: {PROJECT_ROOT}")
print(f"Metadata file: {METADATA_PATH}")

Project Root: f:\Prof. Nick
Metadata file: f:\Prof. Nick\data\raw\Coupon_metadata.csv


In [3]:
'''Cell 3: Load Full Metadata'''
# ==============================================================================
try:
    metadata_df = pd.read_csv(METADATA_PATH)
    metadata_df.columns = metadata_df.columns.str.strip()
    '''
    metadata_df.rename(columns={
        'Coupon': 'coupon_id',
        'Center_X': 'midpoint_x',
        'Center_Y': 'midpoint_y'
    }, inplace=True)
    '''
    
    print("✅ Successfully loaded and processed Coupon_metadata.csv")
    
except FileNotFoundError:
    print(f"❌ ERROR: Metadata file not found at '{METADATA_PATH}'")
    metadata_df = None
except Exception as e:
    print(f"An error occurred while loading metadata: {e}")
    metadata_df = None


✅ Successfully loaded and processed Coupon_metadata.csv


In [ ]:
'''Cell 4: Main Segmentation Loop'''
# ==============================================================================
def segment_all_builds(raw_root, interim_root, full_metadata, box_size, angle_deg):
    if full_metadata is None:
        print("Aborting: Metadata not loaded.")
        return

    try:
        build_pattern = re.compile(r'^Build_(\d+)', re.IGNORECASE)
        build_dirs = sorted([d for d in os.listdir(raw_root) if os.path.isdir(os.path.join(raw_root, d)) and build_pattern.match(d)])
    except FileNotFoundError:
        print(f"❌ Raw data root directory not found: {raw_root}")
        return

    if not build_dirs:
        print("❌ No 'Build_...' subfolders found in 'data/raw/'.")
        return

    print(f"Found {len(build_dirs)} builds to process: {build_dirs}")
    
    for build_dir_name in build_dirs:
        
        match = build_pattern.match(build_dir_name)
        if not match:
            continue
        
        build_num = int(match.group(1))
        build_id_from_folder = f'B{build_num}'
        
        raw_data_dir = os.path.join(raw_root, build_dir_name)
        interim_dir = os.path.join(interim_root, f"{build_id_from_folder}_segmented")
        os.makedirs(interim_dir, exist_ok=True)
        
        print(f"\n--- Starting processing for: {build_id_from_folder} ---")

        build_metadata = full_metadata[full_metadata['Build_id'] == build_id_from_folder].copy()
        
        if build_metadata.empty:
            print(f"⚠️ Warning: No metadata found for {build_id_from_folder}. Skipping this build.")
            continue
            
        build_metadata.set_index('Coupon', inplace=True)
        coupon_midpoint_map = build_metadata[['Center_X', 'Center_Y']].to_dict('index')

        segment_single_build(raw_data_dir, interim_dir, coupon_midpoint_map, box_size, angle_deg)

def segment_single_build(raw_dir, output_dir, midpoints, box_size, angle_deg):
    """
    Processes all layer files for a single build and saves them into
    coupon-specific subdirectories.
    """
    half_box = box_size / 2
    angle_rad = np.deg2rad(-angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

    txt_files = [f for f in os.listdir(raw_dir) if f.lower().endswith(".txt")]
    txt_files.sort(key=lambda f: int(re.search(r'(\d+)', f).group()))
    
    for txt_file in tqdm(txt_files, desc=f"  Segmenting {os.path.basename(raw_dir)}"):
        try:
            layer_num = int(re.search(r'(\d+)', txt_file).group(1))
            
            full_layer_df = pd.read_csv(os.path.join(raw_dir, txt_file), sep='\t', header=0)
            full_layer_df.columns = full_layer_df.columns.str.replace(r"\s*\(mean\)", "", regex=True).str.strip()
            
            cols_to_use = ["Demand X", "Demand Y", "MeltVIEW plasma", "MeltVIEW melt pool", "LaserVIEW"]
            layer_df = full_layer_df[cols_to_use]

            for coupon_id, midpoint in midpoints.items():
                # --- CHANGE: Create a subfolder for each coupon ---
                coupon_output_dir = os.path.join(output_dir, coupon_id)
                os.makedirs(coupon_output_dir, exist_ok=True)
                
                # Use the correct keys ('Center_X', 'Center_Y') from the dictionary
                x_mid, y_mid = midpoint['Center_X'], midpoint['Center_Y']
                
                dx = layer_df["Demand X"] - x_mid
                dy = layer_df["Demand Y"] - y_mid
                x_rot = dx * cos_a - dy * sin_a
                y_rot = dx * sin_a + dy * cos_a

                mask = (
                    (x_rot >= -half_box) & (x_rot <= half_box) &
                    (y_rot >= -half_box) & (y_rot <= half_box)
                )
                
                coupon_zone_df = layer_df[mask]

                if not coupon_zone_df.empty:
                    # Save the segmented file into its coupon-specific folder
                    save_filename = f"layer_{layer_num:04d}.csv" # Simplified filename
                    save_path = os.path.join(coupon_output_dir, save_filename)
                    coupon_zone_df.to_csv(save_path, index=False)
        except Exception as e:
            print(f"    ⚠️ Could not process {txt_file}. Error: {e}")
            
    print(f"✅ Segmentation complete for this build.")

In [5]:
'''Cell 5: Run the Full Automated Process'''
# ==============================================================================
if metadata_df is not None:
    segment_all_builds(RAW_ROOT_DIR, INTERIM_ROOT_DIR, metadata_df, BOX_SIZE, ANGLE_DEG)
else:
    print("Could not run process because metadata was not loaded.")

Found 5 builds to process: ['Build_01_SS316LCubes_26_02_2025', 'Build_02_SS316LCubes_28_02_2025', 'Build_03_SS316LCubes_04_03_2025', 'Build_04_SS316LCubes_05_03_2025', 'Build_05_SS316LCubes_06_03_2025']

--- Starting processing for: B1 ---


  Segmenting Build_01_SS316LCubes_26_02_2025: 100%|██████████| 183/183 [11:38<00:00,  3.81s/it]


✅ Segmentation complete for this build.

--- Starting processing for: B2 ---


  Segmenting Build_02_SS316LCubes_28_02_2025: 100%|██████████| 366/366 [30:16<00:00,  4.96s/it]


✅ Segmentation complete for this build.

--- Starting processing for: B3 ---


...

✅ Segmentation complete for this build.

--- Starting processing for: B4 ---


  Segmenting Build_04_SS316LCubes_05_03_2025: 100%|██████████| 365/365 [38:13<00:00,  6.28s/it]


✅ Segmentation complete for this build.

--- Starting processing for: B5 ---


  Segmenting Build_05_SS316LCubes_06_03_2025: 100%|██████████| 366/366 [39:52<00:00,  6.54s/it]

✅ Segmentation complete for this build.
